In [119]:
import dataiku
import dataikuapi
from dataiku import pandasutils as pdu
import pandas as pd
import logging

In [120]:
user_client = dataiku.api_client()
user_auth_info = user_client.get_auth_info()

In [121]:
design_client = dataiku.api_client()   
dss_groups = design_client.list_groups()
  
group_choices = []
for group in dss_groups:
    group_choices.append({ "value" : group['name'], "label" : group['name']})

In [122]:
group_choices

[{'value': 'administrators', 'label': 'administrators'},
 {'value': 'data_team', 'label': 'data_team'},
 {'value': 'readers', 'label': 'readers'},
 {'value': 'MLOps', 'label': 'MLOps'},
 {'value': 'cc_fraud_project', 'label': 'cc_fraud_project'},
 {'value': 'churn_prediction_project', 'label': 'churn_prediction_project'}]

## Design

In [123]:
design_host = dataiku.api_client().host 
print(design_host)

http://127.0.0.1:20001


In [150]:
design_client = dataikuapi.DSSClient(design_host, "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [151]:
project = design_client.get_project("MKT_CHURN_PREDICTION")   
project_permissions = project.get_permissions()

In [162]:
project_key = "MKT_CHURN_PREDICTION"

In [170]:
#bundle_version_type = config.get("bundle_version_type", "")
bundle_version_type = "minor"

In [190]:
list_bundles = ['MKT_CHURN_PREDICTION_v1_1', 'MKT_CHURN_PREDICTION_v1_2', 'MKT_CHURN_PREDICTION_v1']
#list_bundles = ["test"]
list_bundles = []

In [191]:
list_bundles = sorted(list_bundles)

In [203]:
exported_bundles = project.list_exported_bundles()['bundles']

In [205]:
list_bundles

['MKT_CHURN_PREDICTION_v1_1',
 'MKT_CHURN_PREDICTION_v1_2',
 'MKT_CHURN_PREDICTION_v1']

In [198]:
for bundle in exported_bundles:
    if bundle['state'] == 'BUILT':
        print("ok")
    else:
        print("not ok")

not ok


In [206]:
#version_id = 1
exported_bundles = project.list_exported_bundles()['bundles']
list_bundles = []
for bundles in exported_bundles:
    if (project_key in bundles['bundleId']) and (bundles['state'] == 'BUILT'):
        list_bundles.append(bundles['bundleId'])
list_bundles = sorted(list_bundles)
if len(list_bundles) == 0: 
    if bundle_version_type == "major":
        version_id = 1
    elif bundle_version_type == "minor":
        raise Exception("This is the first time you deploy the project, it should be a major version!") 
elif len(list_bundles) > 0:
    last_deployed_bundles = list_bundles[-1]
    last_version = last_deployed_bundles.split('v')[1].split('_')
    if bundle_version_type == "major":
        version_id = int(last_version[0]) + 1
    elif bundle_version_type == "minor":
        if len(last_version) == 1:
            version_id = last_version[0] + '_1'
        elif len(last_version) == 2:
            version_id = last_version[0] + '_' + str(int(last_version[1]) + 1)

bundle_id = project_key + '_v' + str(version_id)

In [207]:
print(bundle_id)

MKT_CHURN_PREDICTION_v1_3


In [144]:
[permission["group"] for permission in project_permissions \
 if (permission.get("writeProjectContent", False)==True) or (permission.get("admin", False)==True) ]

['MLOps']

In [108]:
plugin = design_client.get_plugin('project-permissions')

In [109]:
settings = plugin.get_plugin_config()

AttributeError: 'DSSPlugin' object has no attribute 'get_plugin_config'

In [97]:
raw_settings = settings.get_raw()

In [101]:
raw_settings['config']['project_domain']

{'OPT': 'Operation',
 'FIN': 'Finance',
 'MKT': 'Marketing',
 'LOG': 'Logistics',
 'PAY': 'Pay'}

In [102]:
dss_groups = design_client.list_groups()
group_choices = []
for group in dss_groups:
    group_choices.append({ "value" : group['name'], "label" : group['name']})

In [103]:
group_choices

[{'value': 'administrators', 'label': 'administrators'},
 {'value': 'data_team', 'label': 'data_team'},
 {'value': 'readers', 'label': 'readers'},
 {'value': 'MLOps', 'label': 'MLOps'},
 {'value': 'cc_fraud_project', 'label': 'cc_fraud_project'},
 {'value': 'churn_prediction_project', 'label': 'churn_prediction_project'}]

In [106]:
curr_log_user = design_client.get_own_user().get_settings().get_raw()["login"]

In [86]:
dss_users = design_client.list_users()
user_info = [user for user in dss_users if user['login'] == curr_log_user]
if 'MLOps' in user_info[0]['groups']:
    logging.info("User {} is in the 'MLOps' group".format(curr_log_user))
else:
    logging.error("User {} is not in the 'MLOps' group".format(curr_log_user))
    raise Exception("User {} is not in the 'MLOps' group".format(curr_log_user))
        

In [107]:
project = design_client.get_project("DSPIPELINEFRAUD")

In [108]:
project_permissions = project.get_permissions().get("permissions")

In [98]:
#bundle_version_type = config.get("bundle_version_type", "")
bundle_version_type = "minor"

In [99]:
#exported_bundles = ["DSPIPELINEFRAUD_v1", "DSPIPELINEFRAUD_v1.1", "DSPIPELINEFRAUD_v1.2", "DSPIPELINEFRAUD_v2", "DSPIPELINEFRAUD_v2.1"]

In [100]:
#version_id = 1
exported_bundles = project.list_exported_bundles()['bundles']
list_bundles = []
for bundles in exported_bundles:
    list_bundles.append(bundles['bundleId'])
if len(list_bundles) == 0: 
    if bundle_version_type == "major":
        version_id = 1
    elif bundle_version_type == "minor":
        raise Exception("This is the first time you deploy the project, it should be a major version!") 
elif len(list_bundles) > 0:
    last_deployed_bundles = list_bundles[-1]
    last_version = last_deployed_bundles.split('v')[1].split('_')
    if bundle_version_type == "major":
        version_id = int(last_version[0]) + 1
    elif bundle_version_type == "minor":
        if len(last_version) == 1:
            version_id = last_version[0] + '_1'
        elif len(last_version) == 2:
            version_id = last_version[0] + '_' + str(int(last_version[1]) + 1)

bundle_id = project_key + '_v' + str(version_id)

In [101]:
bundle_id

'DSPIPELINEFRAUD_v1_4'

In [92]:
list_bundles = list(set(list_bundles))

In [93]:
list_bundles

['DSPIPELINEFRAUD_v1_3',
 'DSPIPELINEFRAUD_v1_2',
 'DSPIPELINEFRAUD_v1',
 'DSPIPELINEFRAUD_v1_1']

## Automation QA

In [110]:
host = "http://localhost:15000/"
auto_client = dataikuapi.DSSClient(host, "Da1cTKi0eO3VMUUvCq7TuIfSHRBzY3OA") 
project = auto_client.get_project("DSPIPELINEFRAUD")

In [111]:
deployed_project = auto_client.get_project("mkt_churn_model")   
deployed_project_permissions = deployed_project.get_permissions()

In [113]:
deployed_project_permissions

{'owner': 'mkt_admin',
 'permissions': [{'group': 'MLOps',
   'admin': False,
   'readProjectContent': True,
   'writeProjectContent': True,
   'exportDatasetsData': False,
   'readDashboards': True,
   'writeDashboards': True,
   'moderateDashboards': True,
   'shareToWorkspaces': False,
   'runScenarios': True,
   'manageDashboardAuthorizations': False,
   'manageExposedElements': False,
   'manageAdditionalDashboardUsers': False,
   'executeApp': False},
  {'group': 'admin',
   'admin': False,
   'readProjectContent': True,
   'writeProjectContent': True,
   'exportDatasetsData': False,
   'readDashboards': True,
   'writeDashboards': True,
   'moderateDashboards': True,
   'shareToWorkspaces': False,
   'runScenarios': True,
   'manageDashboardAuthorizations': False,
   'manageExposedElements': False,
   'manageAdditionalDashboardUsers': False,
   'executeApp': False}],
 'additionalDashboardUsers': {'users': []},
 'dashboardAuthorizations': {'allAuthorized': False, 'authorizations'

In [8]:
project.list_imported_bundles()

{'bundles': [{'importState': {'importedBy': 'admin',
    'importedOn': '2022-05-25T15:57:24.041+0200'},
   'bundleId': 'DSPIPELINEFRAUD_v1',
   'contentSummary': {'configContent': {'datasets': 24,
     'recipes': 16,
     'savedModels': 1,
     'modelEvaluationStores': 0,
     'managedFolders': 1,
     'scenarios': 0,
     'analysis': 2,
     'jupyterNotebooks': 3,
     'sqlNotebooks': 0,
     'insights': 16,
     'dashboards': 3},
    'includedDatasets': [{'name': 'transactions_2018',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 5404878},
     {'name': 'transactions_2017',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 7423251},
     {'name': 'merchant_info',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 3362746},
     {'name': 'cardholder_info',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 2543734}],
    'includedSavedModels': [],
    'i

## Automation PROD

In [9]:
host = "http://localhost:15500/"
auto_client = dataikuapi.DSSClient(host, "rAo6O1aluJBrMAJEf8vD3vyze3jzgVjF") 
project = auto_client.get_project("DSPIPELINEFRAUD")

In [10]:
project.list_imported_bundles()

{'bundles': [{'importState': {'importedBy': 'admin',
    'importedOn': '2022-05-23T11:42:35.804+0200'},
   'bundleId': 'DSPIPELINEFRAUD_v1',
   'contentSummary': {'configContent': {'datasets': 24,
     'recipes': 16,
     'savedModels': 1,
     'modelEvaluationStores': 0,
     'managedFolders': 1,
     'scenarios': 0,
     'analysis': 2,
     'jupyterNotebooks': 3,
     'sqlNotebooks': 0,
     'insights': 16,
     'dashboards': 3},
    'includedDatasets': [{'name': 'transactions_2018',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 5404878},
     {'name': 'transactions_2017',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 7423251},
     {'name': 'merchant_info',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 3362746},
     {'name': 'cardholder_info',
      'type': 'UploadedFiles',
      'usedTransferMethod': 'UPLOAD',
      'size': 2543734}],
    'includedSavedModels': [],
    'i

## Deployer

In [109]:
writers = [permission["group"] for permission in project_permissions if permission.get("writeProjectContent", False)==True]

In [156]:
writers = ['MLOps']

['MLOps', 'cc_fraud_project']

In [144]:
project_deployer_client = dataikuapi.DSSClient('http://localhost:20000/', "FJk7Jffd8TStmT4Zvx9efC6G5mYZusec") 

In [145]:
project_owner = project.get_permissions().get("owner")
published_project = project_deployer_client.get_project("DSPIPELINEFRAUD") 

In [146]:
published_project

In [147]:
published_project_settings = published_project.get_settings()
raw_permissions = published_project_settings.get_raw()

In [150]:
raw_permissions.keys()

dict_keys(['projectStatus', 'imgPattern', 'showInitials', 'settings', 'exposedObjects', 'metrics', 'metricsChecks', 'bundleExporterSettings', 'projectAppType'])

In [125]:
logging.info("Project owner will again be {}".format(curr_log_user))

In [151]:
# Set the new project owner on the Deployer
raw_permissions['owner'] = curr_log_user

In [152]:
writers

['MLOps', 'cc_fraud_project']

In [154]:
for writer in writers:
    permission = {
            'admin': True, 
            'deploy': True, 
            'group': writer, 
            'read': True, 
            'write': True
    }
    raw_permissions['permissions'].append(permission)


KeyError: 'permissions'

In [133]:
raw_permissions['permissions'] = permission

In [135]:
        
    # this part deduplicates any permissions given to the same group.
permissions = { each['group'] : each for each in raw_permissions['permissions'] }.values()

TypeError: string indices must be integers

In [140]:
for each in raw_permissions['permissions']:
    print(each)
    each['group'] : each

admin
deploy
group
read
write


In [139]:
raw_permissions

'write'

In [0]:
list_permissions = list(permissions)
raw_permissions['permissions'] = list_permissions
published_project_settings.save()

## Plugin settings

In [14]:
from dataiku.customrecipe import *

get_plugin_config()

TypeError: the JSON object must be str, bytes or bytearray, not 'NoneType'

In [19]:
template_libraries = "matplotlib,seaborn"
template_libraries = template_libraries.replace(",",'\n')
print(template_libraries)

matplotlib
seaborn


In [214]:
permissions = [{"admin": False,
                          "executeApp": False,
                          "exportDatasetsData": False,
                          "group": 'MLOps',
                          "manageAdditionalDashboardUsers": False,
                          "manageDashboardAuthorizations": False,
                          "manageExposedElements": False,
                          "moderateDashboards": True,
                          "readDashboards": True,
                          "readProjectContent": True,
                          "runScenarios": True,
                          "writeDashboards": True,
                          "writeProjectContent": True
                         }, {"admin": False,
                          "executeApp": False,
                          "exportDatasetsData": False,
                          "group": 'user1',
                          "manageAdditionalDashboardUsers": False,
                          "manageDashboardAuthorizations": False,
                          "manageExposedElements": False,
                          "moderateDashboards": True,
                          "readDashboards": True,
                          "readProjectContent": True,
                          "runScenarios": True,
                          "writeDashboards": True,
                          "writeProjectContent": True
                         }]

In [212]:
permissions_dict = {}
for each in permissions:
    print(each.keys())
    if 'group' in each.keys():
        permissions_dict['group']: each
    elif 'user' in each.keys():
        permissions_dict['user']: each

dict_keys(['admin', 'executeApp', 'exportDatasetsData', 'group', 'manageAdditionalDashboardUsers', 'manageDashboardAuthorizations', 'manageExposedElements', 'moderateDashboards', 'readDashboards', 'readProjectContent', 'runScenarios', 'writeDashboards', 'writeProjectContent'])
dict_keys(['admin', 'executeApp', 'exportDatasetsData', 'user', 'manageAdditionalDashboardUsers', 'manageDashboardAuthorizations', 'manageExposedElements', 'moderateDashboards', 'readDashboards', 'readProjectContent', 'runScenarios', 'writeDashboards', 'writeProjectContent'])


In [210]:
each

{'admin': False,
 'executeApp': False,
 'exportDatasetsData': False,
 'user': 'user1',
 'manageAdditionalDashboardUsers': False,
 'manageDashboardAuthorizations': False,
 'manageExposedElements': False,
 'moderateDashboards': True,
 'readDashboards': True,
 'readProjectContent': True,
 'runScenarios': True,
 'writeDashboards': True,
 'writeProjectContent': True}

In [215]:
permissions = { each['group'] : each for each in permissions }.values()

In [219]:
{ each['group'] : each for each in permissions }

{'MLOps': {'admin': False,
  'executeApp': False,
  'exportDatasetsData': False,
  'group': 'MLOps',
  'manageAdditionalDashboardUsers': False,
  'manageDashboardAuthorizations': False,
  'manageExposedElements': False,
  'moderateDashboards': True,
  'readDashboards': True,
  'readProjectContent': True,
  'runScenarios': True,
  'writeDashboards': True,
  'writeProjectContent': True},
 'user1': {'admin': False,
  'executeApp': False,
  'exportDatasetsData': False,
  'group': 'user1',
  'manageAdditionalDashboardUsers': False,
  'manageDashboardAuthorizations': False,
  'manageExposedElements': False,
  'moderateDashboards': True,
  'readDashboards': True,
  'readProjectContent': True,
  'runScenarios': True,
  'writeDashboards': True,
  'writeProjectContent': True}}

In [216]:
list_permissions = list(permissions)

In [217]:
list_permissions

[{'admin': False,
  'executeApp': False,
  'exportDatasetsData': False,
  'group': 'MLOps',
  'manageAdditionalDashboardUsers': False,
  'manageDashboardAuthorizations': False,
  'manageExposedElements': False,
  'moderateDashboards': True,
  'readDashboards': True,
  'readProjectContent': True,
  'runScenarios': True,
  'writeDashboards': True,
  'writeProjectContent': True},
 {'admin': False,
  'executeApp': False,
  'exportDatasetsData': False,
  'group': 'user1',
  'manageAdditionalDashboardUsers': False,
  'manageDashboardAuthorizations': False,
  'manageExposedElements': False,
  'moderateDashboards': True,
  'readDashboards': True,
  'readProjectContent': True,
  'runScenarios': True,
  'writeDashboards': True,
  'writeProjectContent': True}]